In [30]:
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from pythreejs import *
import cv2

In [31]:
# File paths
calibration_file_path = "kitti_object_vis-master/data/object/training/calib/000000.txt"
image_file_path = "kitti_object_vis-master/data/object/training/image_2/000000.png"
lidar_data_path = "kitti_object_vis-master/data/object/training/velodyne/000000.bin"

stream = open(image_file_path, "rb")
bytes = bytearray(stream.read())
numpyarray = np.asarray(bytes, dtype=np.uint8)
image_color = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)
image_color = image_color[:, :, (2, 1, 0)]  # BGR -> RGB
print(image_color.shape)
image_color_row = image_color.shape[0]
image_color_column = image_color.shape[1]

(370, 1224, 3)


In [32]:
# Read calibration data
with open(calibration_file_path, 'r') as fid:
    lines = fid.readlines()
    for i in range(7):
        lines[i] = lines[i].strip().split(' ')
        del lines[i][0]
        lines[i] = ' '.join(lines[i])

    P0 = np.fromstring(lines[0], dtype=float, sep=' ')
    P1 = np.fromstring(lines[1], dtype=float, sep=' ')
    P2 = np.fromstring(lines[2], dtype=float, sep=' ')
    P3 = np.fromstring(lines[3], dtype=float, sep=' ')
    R0_rect = np.fromstring(lines[4], dtype=float, sep=' ')
    Tr_velo_to_cam = np.fromstring(lines[5], dtype=float, sep=' ')
    Tr_imu_to_velo = np.fromstring(lines[6], dtype=float, sep=' ')
    Tr_cam_to_road = np.fromstring(lines[7], dtype=float, sep=' ')

In [33]:
#Read LiDAR data
data = np.fromfile(lidar_data_path, dtype=np.float32).reshape(-1, 4)
# Perform LiDAR-to-Image mapping
R0 = R0_rect.reshape(3,3)
Tr = Tr_velo_to_cam.reshape(3,4)
p2 = P2.reshape(3,4)
XYZ = data[:,:3].T

XYZ1 = np.vstack((XYZ, np.ones(data.shape[0])))
XYZ1 = np.dot(R0, np.dot(Tr, XYZ1))
XYZ1 = np.vstack((XYZ1, np.ones(XYZ1.T.shape[0])))
xy1 = np.dot(p2,XYZ1)

In [34]:
p2=P2.reshape(3,4)

#separate K, R, and T through the decomposeProjectionMatrix function of openCV.
k2,r2,t2,_,_,_,_= cv2.decomposeProjectionMatrix(p2)

#The reason why there are four elements of t2 is that when decompose is performed, T is cut off based on the size of 4X4.
#Therefore, the last element is unnecessary.
print(t2)
t2=t2[:3,0]

[[-0.06035061]
 [ 0.00175693]
 [-0.00497187]
 [ 0.99816331]]


In [35]:
weight = xy1[2, :]
x=xy1[0,:]/weight
y=xy1[1,:]/weight
k = np.where(weight > 0)[0]
valid_xy1=xy1.T[k]

#PCD information in world coordinate system
valid_xy1=np.dot(np.linalg.inv(k2),valid_xy1.T)
valid_xy1=valid_xy1.T-t2
valid_xy1=np.dot(np.linalg.inv(r2),valid_xy1.T)
depth_camera02=valid_xy1.T

x=x[k].reshape(-1,1)
y=y[k].reshape(-1,1)
w=weight[k].reshape(-1,1)
xy=np.hstack((x,y))

xy_index11=np.where(xy[:,0]<image_color_column)
xy_index12=np.where(xy[:,0]>=0)
xy_index21=np.where(xy[:,1]<image_color_row)
xy_index22=np.where(xy[:,1]>=0)
xy_index1=np.intersect1d(xy_index11,xy_index12)
xy_index2=np.intersect1d(xy_index21,xy_index22)

#xy_index: Index of point clouds that come within the scope of the camera's view
xy_index=np.intersect1d(xy_index1,xy_index2)
reshaped_xy=xy[xy_index]
print(reshaped_xy.shape)

(20285, 2)


In [36]:
RGB_to_world_coordinate=np.zeros((image_color_row*image_color_column,2)).reshape(-1,2)

#Save the (x,y) coordinates of the picture in matrix form in pixel coordinate
for i in range(image_color_row):
    for j in range(image_color_column):
        RGB_to_world_coordinate[i*1224+j,0]=i
        RGB_to_world_coordinate[i*1224+j,1]=j
image_color=image_color.reshape(-1,3)
print(image_color.shape)

#The (x,y) coordinates of the pixel and the (r,g,b) information are hstacked.
RGB_tmp=np.hstack((RGB_to_world_coordinate,image_color))
print(RGB_tmp)

(452880, 3)
[[0.000e+00 0.000e+00 1.700e+01 2.200e+01 1.800e+01]
 [0.000e+00 1.000e+00 1.600e+01 2.400e+01 2.800e+01]
 [0.000e+00 2.000e+00 2.500e+01 2.900e+01 3.300e+01]
 ...
 [3.690e+02 1.221e+03 2.200e+01 3.000e+01 3.700e+01]
 [3.690e+02 1.222e+03 2.200e+01 2.900e+01 3.800e+01]
 [3.690e+02 1.223e+03 2.000e+01 3.000e+01 3.500e+01]]


In [37]:
#Since each coordinate information of a pixel is an integer, all points belonging to one pixel may have the same color.
int_xy=reshaped_xy.astype(int)
RGB=np.zeros((len(int_xy.T[0]),3))
for i in range(len(int_xy.T[0])):
    index_RGB_tmp=int_xy[i,1]*image_color_column+int_xy[i,0]
    RGB[i]=RGB_tmp[index_RGB_tmp,2:6]

In [38]:
image_point_tmp=np.vstack((reshaped_xy.T,np.ones(reshaped_xy.T[0].shape)))

image_point_tmp=np.dot(np.linalg.inv(k2),image_point_tmp)
image_point_tmp=image_point_tmp.T-t2
image_point_tmp=np.dot(np.linalg.inv(r2),image_point_tmp.T)

#When image_point is displayed, The point cloud is taken in the form of a photo.
image_point=np.hstack((image_point_tmp.T,RGB))
#When image_point2 is displayed, The point cloud is displayed as 3D PCD map.
image_point2=np.hstack((depth_camera02[xy_index],RGB))

In [39]:
Fov= 2*math.atan(p2[1,1]/p2[0,0])*(180/pi)
print(Fov)

90.0


In [40]:
normalized_RGB_color = image_point2 / 255.0
geometry = BufferGeometry(
     attributes={
        'position': BufferAttribute(image_point2[:,:3], normalized=False),
        'color': BufferAttribute(normalized_RGB_color[:,3:6], normalized=True)
     }
 )

material = PointsMaterial(size=0.1, vertexColors='VertexColors')
point=Points(geometry=geometry, material=material)

camera = PerspectiveCamera(up=[0,0,1], position=[0,-10,-10], near=0.01, aspect=400/300)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()
scene=Scene(children=[point,camera,key_light,ambient_light], background=None)
scene.add(AxesHelper(size=3))

c:\Users\loyid\AppData\Local\Programs\Python\Python311\Lib\site-packages\pythreejs\traits.py:257: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


In [41]:
lidar_position=np.dot(R0,np.dot(Tr,[0,0,0,1]))
print(lidar_position)

#PointLightHelper for expressing lidar sensor's position in the world coordinate system
point_light = PointLight(color="#ffffff", intensity=1, distance=100)
lidar_position=(lidar_position[0],lidar_position[1],lidar_position[2])
point_light_helper = PointLightHelper(point_light, position=lidar_position, distance=0.1, sphereSize=0.1)
scene.add(point_light)
scene.add(point_light_helper)

[-0.02236671 -0.05967891 -0.332549  ]


In [42]:
cam_position=[0,0,0]
cam_position=np.dot(np.linalg.inv(k2),cam_position)
cam_position=cam_position-t2
cam_position=np.dot(np.linalg.inv(r2),cam_position)
print(cam_position)

near_condition=np.where(image_point2[:,0]>=0)
near=np.min(image_point2[near_condition,0])
far=np.max(image_point2[:,0])

[ 0.06035061 -0.00175693  0.00497187]


In [43]:
#image_point's depth: Focal length
print(np.min(image_point[:,2]))
print(np.max(image_point[:,2]))
near=np.max(image_point[:,2])

1.0049718674293782
1.0049718674293782


In [44]:
cam=PerspectiveCamera(up=[0,0,-1], position=(cam_position[0],cam_position[1],cam_position[2]), aspect=image_color_row/image_color_column, fov=Fov, near=near, far=far)
camera_helper = CameraHelper(cam)
camera_helper.rotateX(math.pi)
camera_helper.rotateZ(math.pi/2)
scene.add(camera_helper)

In [45]:
point_light = PointLight(color="#ffffff", intensity=1, distance=100)
scene.add(point_light)

#scene.add(camera_helper)
controller=OrbitControls(controlling=camera)
renderer=Renderer(camera=camera, scene=scene, controls=[controller],width=800,height=600)
display(renderer)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, near=0.01, position=(0.0, -10.0, -10.0), projecti…